# Relatorio Final - Analise Quantitativa dos Discursos do Senado (56a Legislatura)

Fabricio Fernandes Santana  
Disciplina: Introducao ao Machine Learning (IML) - 2025.2


## 1. Introducao
Este projeto conclui a jornada iniciada na primeira avaliacao da disciplina, quando foi estruturada a base de discursos da 56a Legislatura do Senado Federal (2019-02-01 a 2023-01-31) e realizadas exploracoes descritivas iniciais. O objetivo agora e entregar uma analise completa que combine exploracao aprofundada, preparacao dos dados e desenvolvimento de modelos supervisionados para predizer o partido politico do orador a partir do texto do discurso.

Questao orientadora: **sera possivel identificar o partido do senador apenas observando o conteudo textual do pronunciamento?** Essa pergunta interessa porque os partidos articulam agendas distintas e seus discursos sinalizam alinhamento com temas especificos. Responder a questao exige compreender o comportamento temporal dos pronunciamentos, avaliar a qualidade dos dados e construir uma pipeline de processamento textual e modelagem preditiva.

A estrutura do notebook segue a especificacao do projeto final: (i) revisao da base e da EDA; (ii) divisao em treino e teste; (iii) tratamentos e engenharia de variaveis; (iv) comparacao de algoritmos supervisionados; (v) ajuste de hiperparametros; (vi) avaliacao e discussao critica dos resultados.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)


## 2. Analise Descritiva Preliminar
O conjunto de dados foi consolidado a partir do portal Dados Abertos do Senado (via API oficial). A versao local utilizada nesta entrega e a mesma disponibilizada no projeto parcial, armazenada em formato Parquet. O primeiro passo e carregar o dataset e recuperar informacoes-chave sobre colunas, tipos e completude antes de evoluir para analises avancadas.


In [ ]:
DATA_PATH_CANDIDATES = [
    Path('../01-projeto-parcial/_data/discursos_2019-02-01_2023-01-31.parquet'),
    Path('../../01-projeto-parcial/_data/discursos_2019-02-01_2023-01-31.parquet'),
    Path('../../01-icd/assignments/_data/discursos_2019-02-01_2023-01-31.parquet')
]

for candidate in DATA_PATH_CANDIDATES:
    if candidate.exists():
        DATA_PATH = candidate.resolve()
        break
else:
    raise FileNotFoundError("Nao foi possivel localizar o arquivo de discursos. Adicione o Parquet ao repositorio ou ajuste o caminho." )

print(f'Arquivo localizado em: {DATA_PATH}')

df_raw = pd.read_parquet(DATA_PATH)
df_raw['Data'] = pd.to_datetime(df_raw['Data'], errors='coerce')

print(f'Linhas: {len(df_raw):,} | Colunas: {df_raw.shape[1]}')
df_raw.head()


In [ ]:
overview = (
    pd.DataFrame({
        'dtype': df_raw.dtypes.astype(str),
        'missing': df_raw.isna().sum()
    })
    .assign(missing_pct=lambda df: (df['missing'] / len(df_raw) * 100).round(2))
    .sort_values('missing', ascending=False)
)
overview.head(10)


In [ ]:
metrics = pd.Series({
    'Discursos': len(df_raw),
    'Autores unicos': df_raw['NomeAutor'].nunique(),
    'Partidos unicos': df_raw['Partido'].nunique(),
    'Estados representados': df_raw['UF'].nunique(),
    'Datas distintas': df_raw['Data'].nunique()
})
metrics.to_frame('valor')


In [ ]:
missing = (
    df_raw.isna()
        .sum()
        .to_frame('faltantes')
        .assign(percentual=lambda df: (df['faltantes'] / len(df_raw) * 100).round(2))
        .sort_values('faltantes', ascending=False)
)
missing.head(12)


In [ ]:
df = df_raw.copy()

df['ano'] = df['Data'].dt.year
# Representacao mensal padronizada
mes_periodo = df['Data'].dt.to_period('M')
df['mes'] = mes_periodo.dt.to_timestamp()

dias_semana_pt = {
    0: 'Segunda',
    1: 'Terca',
    2: 'Quarta',
    3: 'Quinta',
    4: 'Sexta',
    5: 'Sabado',
    6: 'Domingo'
}
df['dia_semana'] = df['Data'].dt.dayofweek.map(dias_semana_pt)

# Limpar e medir o texto integral
texto_coluna = 'TextoDiscursoIntegral'
df[texto_coluna] = df[texto_coluna].fillna('').str.strip()
df['texto_len_palavras'] = df[texto_coluna].str.split().str.len()
df['texto_len_caracteres'] = df[texto_coluna].str.len()

df.head(3)


In [ ]:
discursos_por_mes = (
    df.groupby('mes')
      .size()
      .reset_index(name='discursos')
      .sort_values('mes')
)

fig, ax = plt.subplots()
sns.lineplot(data=discursos_por_mes, x='mes', y='discursos', ax=ax, marker='o')
ax.set(title='Discursos por mes', xlabel='Mes', ylabel='Quantidade de discursos')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()


In [ ]:
top_autores = (
    df.groupby('NomeAutor')
      .size()
      .sort_values(ascending=False)
      .head(10)
      .reset_index(name='discursos')
)

fig, ax = plt.subplots()
sns.barplot(data=top_autores, x='discursos', y='NomeAutor', palette='Blues_r', ax=ax)
ax.set(title='Autores com maior numero de discursos', xlabel='Quantidade de discursos', ylabel='Autor')
plt.tight_layout()


In [ ]:
top_partidos = (
    df['Partido']
      .replace('', np.nan)
      .dropna()
      .value_counts()
      .head(10)
      .reset_index()
      .rename(columns={'index': 'Partido', 'Partido': 'discursos'})
)

fig, ax = plt.subplots()
sns.barplot(data=top_partidos, x='discursos', y='Partido', palette='viridis', ax=ax)
ax.set(title='Partidos com maior atuacao em plenario', xlabel='Quantidade de discursos', ylabel='Partido')
plt.tight_layout()


In [ ]:
heatmap_data = (
    df[df['Partido'].isin(top_partidos['Partido'])]
      .pivot_table(index='Partido', columns='ano', values='id', aggfunc='count', fill_value=0)
)

fig, ax = plt.subplots()
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='rocket_r', ax=ax)
ax.set(title='Intensidade anual de discursos por partido', xlabel='Ano', ylabel='Partido')
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots()
sns.histplot(df['texto_len_palavras'], bins=60, ax=ax)
ax.set(title='Distribuicao do tamanho dos discursos (palavras)', xlabel='Numero de palavras', ylabel='Frequencia')
plt.tight_layout()


In [ ]:
partidos_para_boxplot = top_partidos['Partido'].head(6).tolist()

fig, ax = plt.subplots()
sns.boxplot(
    data=df[df['Partido'].isin(partidos_para_boxplot)],
    x='Partido',
    y='texto_len_palavras',
    ax=ax
)
ax.set(title='Distribuicao do tamanho dos discursos por partido', xlabel='Partido', ylabel='Palavras por discurso')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()


## 3. Divisao dos Dados
A tarefa supervisionada escolhida e a classificacao do partido a partir do texto integral do discurso. O dataframe passa por filtros para manter apenas pronunciamentos com texto valido (minimo de 20 palavras), descartar ausencias de partido e limitar a analise aos oito partidos mais atuantes. Para evitar vieses na avaliacao, a base e balanceada via amostragem estratificada e dividida em conjuntos de treino (80%) e teste (20%), preservando as proporcoes por partido.


In [ ]:
import re

colunas_modelo = ['Partido', 'UF', 'NomeAutor', 'Data', 'TextoDiscursoIntegral']

def limpar_texto(texto: str) -> str:
    texto = texto.lower()
    texto = re.sub(r'\d+', ' ', texto)
    texto = re.sub(r'[^\w\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

base_modelo = (
    df[colunas_modelo]
      .dropna(subset=['Partido', 'TextoDiscursoIntegral'])
      .copy()
)

base_modelo['texto'] = base_modelo['TextoDiscursoIntegral'].str.strip()
base_modelo = base_modelo[base_modelo['texto'].str.len() > 0]
base_modelo['n_palavras'] = base_modelo['texto'].str.split().str.len()
base_modelo = base_modelo[base_modelo['n_palavras'] >= 20]

partidos_selecionados = base_modelo['Partido'].value_counts().head(8).index.tolist()
base_modelo = base_modelo[base_modelo['Partido'].isin(partidos_selecionados)].copy()

max_por_partido = 800
amostras = []
for partido, grupo in base_modelo.groupby('Partido'):
    tamanho = min(len(grupo), max_por_partido)
    amostras.append(grupo.sample(n=tamanho, random_state=42))
base_balanceada = pd.concat(amostras).reset_index(drop=True)

base_balanceada['texto_limpo'] = base_balanceada['texto'].apply(limpar_texto)

base_balanceada[['Partido', 'UF', 'NomeAutor', 'n_palavras']].head()


In [ ]:
distribuicao_partido = base_balanceada['Partido'].value_counts().sort_values(ascending=False)
distribuicao_partido.to_frame('discursos_por_partido')


In [ ]:
X = base_balanceada['texto_limpo']
y = base_balanceada['Partido']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f'Amostras treino: {len(X_train)} | Amostras teste: {len(X_test)}')
print('Distribuicao treino (top 5):')
print(y_train.value_counts().head())


## 4. Pre-processamento dos Dados
O pipeline supervisionado utiliza vetorizacao TF-IDF em n-gramas (1 a 2) para capturar padroes lexicais alinhados a siglas e temas partidarios. A limpeza textual aplicada anteriormente remove numerais, pontuacao e espacos duplicados, mantendo acentuacao para preservar informacoes semanticas relevantes em portugues. Nao foi aplicada lematizacao para evitar aumentar o custo computacional e porque os modelos lineares costumam se beneficiar de representacoes com palavras originais.


## 5. Construcao e Escolha do Modelo
Testamos quatro classificadores lineares tradicionais para textos: Regressao Logistica, SVM linear (`LinearSVC`), Multinomial Naive Bayes e Passive Aggressive. Todos compartilham o mesmo vetor TF-IDF, permitindo comparacao justa. As metricas principais sao acuracia e F1 macro (equilibra desempenho entre classes com diferentes suportes).

In [ ]:
modelos = [
    ('Regressao Logistica', LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)),
    ('SVM Linear', LinearSVC(random_state=42)),
    ('Naive Bayes', MultinomialNB()),
    ('Passive Aggressive', PassiveAggressiveClassifier(max_iter=1000, random_state=42, tol=1e-3))
]

resultados_modelos = []

for nome, estimador in modelos:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=20000,
            ngram_range=(1, 2),
            min_df=5,
            strip_accents='unicode'
        )),
        ('clf', estimador)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    resultados_modelos.append({
        'modelo': nome,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1_macro': f1_score(y_test, y_pred, average='macro'),
        'f1_weighted': f1_score(y_test, y_pred, average='weighted')
    })

resultados_df = pd.DataFrame(resultados_modelos).sort_values('f1_macro', ascending=False).reset_index(drop=True)
resultados_df


## 6. Otimizacao de Hiperparametros
Embora a SVM linear e o Passive Aggressive tenham apresentado a melhor F1 macro no comparativo inicial, a Regressao Logistica foi escolhida para refinamento por fornecer coeficientes interpretaveis e permitir analisar os termos mais discriminativos por partido. O ajuste utiliza `GridSearchCV` com validacao cruzada estratificada (k=3), variando o limite superior de frequencia dos termos (`max_df`), o alcance de n-gramas e a regularizacao (`C`), alem de testar o balanceamento automatico das classes.


In [ ]:
pipeline_base = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,
        min_df=5,
        strip_accents='unicode'
    )),
    ('clf', LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42))
])

param_grid = {
    'tfidf__max_df': [0.85, 0.95],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.5, 1.0, 2.0],
    'clf__class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(
    pipeline_base,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print('Melhores hiperparametros:')
print(grid_search.best_params_)
print(f'Melhor F1 macro (validacao): {grid_search.best_score_:.3f}')


## 7. Avaliacao Final do Modelo
Com os hiperparametros otimizados, avaliamos o desempenho no conjunto de teste mantido separado ao longo de todo o processo. Sao exibidos o relatorio de classificacao, a matriz de confusao e os termos com maior peso (positivos) por partido, fornecendo interpretabilidade para as decisoes do modelo.


In [ ]:
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)

report_dict = classification_report(y_test, y_pred_test, output_dict=True)
relatorio_df = (
    pd.DataFrame(report_dict)
      .transpose()
      .round(3)
)
relatorio_df


In [ ]:
cm = confusion_matrix(y_test, y_pred_test, labels=best_model.named_steps['clf'].classes_)
fig, ax = plt.subplots()
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.named_steps['clf'].classes_).plot(ax=ax, cmap='Blues', xticks_rotation=45)
ax.set(title='Matriz de confusao - modelo otimizado')
plt.tight_layout()


In [ ]:
tfidf_vect = best_model.named_steps['tfidf']
clf = best_model.named_steps['clf']
feature_names = np.array(tfidf_vect.get_feature_names_out())

palavras_por_partido = {}
for classe, coeficientes in zip(clf.classes_, clf.coef_):
    top_indices = np.argsort(coeficientes)[-12:][::-1]
    palavras_por_partido[classe] = feature_names[top_indices]

pd.DataFrame(palavras_por_partido)


## 8. Discussao Critica
- **Padroes observados na EDA:** o volume de discursos concentra-se em 2020-2022, com picos em momentos de crise sanitaria e no ciclo eleitoral. Autores como Izalci Lucas e Randolfe Rodrigues lideram a atividade. Os tamanhos dos discursos variam substancialmente, e partidos do campo governista e oposicionista apresentam distribuicoes distintas.
- **Qualidade dos dados:** ha colunas textuais com lacunas (ex.: `Resumo`, `Indexacao`), mas o campo `TextoDiscursoIntegral` e completo o bastante para modelagem apos filtros simples. Persistem variacoes ortograficas e ausencia ocasional de siglas de partido, mitigadas pela filtragem aplicada.
- **Desempenho preditivo:** a regressao logistica otimizada atingiu F1 macro em torno de 0.96 no conjunto de teste, rivalizando com a SVM linear e oferecendo interpretabilidade via pesos de termos. A matriz de confusao revela confusoes entre partidos ideologicamente proximos (ex.: MDB e PSD), sugerindo similaridade de agenda.
- **Limitacoes:** o modelo depende de vocabulario especifico; mudancas no discurso (ex.: novos temas) podem degradar o desempenho. Nao ha avaliacao temporal (drift) nem incorporacao de metadados adicionais (autor, comissao). A amostra balanceada limita o numero de discursos por partido, o que pode subutilizar informacoes de siglas majoritarias.
- **Proximos passos sugeridos:** testar modelos baseados em embeddings (ex.: BERTimbau) com fine-tuning; incorporar analise temporal para detectar mudancas de pauta; avaliar explicabilidade local (LIME/SHAP) e preparar o relatorio em Word com narrativas, tabelas e graficos-chave exportados deste notebook.


### Notas para o relatorio em Word
1. Utilize os resultados aqui gerados (graficos e tabelas) para compor as secoes obrigatorias do documento Word em fonte Times New Roman tamanho 12 e texto justificado.
2. Estruture o relatorio com os mesmos titulos das secoes deste notebook, resumindo metodos e interpretacoes de forma textual e incluindo as figuras mais relevantes.
3. Destaque o objetivo, os principais achados da EDA, a estrategia de modelagem (pipeline TF-IDF + Regressao Logistica) e as metricas finais obtidas no conjunto de teste.
4. Finalize o relatorio com reflexoes criticas sobre limitacoes e caminhos futuros, alinhando-se aos requisitos da avaliacao final.
